In [1]:
import numpy as np

import importlib

In [ ]:
def diffuse_cuts(mc):
    """Applies diffuse dataset cuts on a given monte-carlo data.

    Parameters
    ----------
    mc : str | numpy record ndarray
        Monte-carlo data.

    Returns
    -------
    mc_dc : numpy record ndarray
        Monte-carlo data after diffuse dataset cuts.
    """
    mc_dc = mc[(mc['trueDec'] > np.radians(-5)) &
               (np.log10(mc['trueE']) < 8.0) &
               (mc['sigmaOK'] == 0)]
    return mc_dc

In [2]:
plotter_wkde = importlib.import_module('plotter_wkde_2')

#gammas = np.concatenate(([1.5], np.linspace(2.0, 3.0, 11)))
gammas = [2.0]

logE_list = [2.5]
sigma_p_list = [0.5]

base_path = '/home/ge56lag'

for gamma in gammas:
    spatial_KDE_path = '{}/Software/KDE/output/sig_psi_E/adaptive_kd_plotter_wkde_gamma_{}_phi0_1.0/pdf/sig_psi_E.pkl'.format(base_path, gamma)
    norm_KDE_path = '{}/Software/KDE/output/sig_E/adaptive_kd_plotter_wkde_gamma_{}_phi0_1.0/pdf/sig_E.pkl'.format(base_path, gamma)

    spatial_pdf, mc = plotter_wkde.setup(base_path, spatial_KDE_path, norm_KDE_path)
    
    mc = diffuse_cuts(mc)

    for logE, sigma_p in zip(logE_list, sigma_p_list):
        plotter_wkde.make_plot(spatial_pdf, mc, logE, sigma_p, gamma=gamma, delta_sigma=0.2, show_quantile=False, output_dir='output_Hans/RGI')

Welcome to JupyROOT 6.14/04


INFO:rootpy.plotting.style] using matplotlib style 'ATLAS'
plotter_wkde_2.py:41: RuntimeWarning: divide by zero encountered in divide
  spatial_KDE_vals = spatial_KDE['pdf_vals']/norm_KDE['pdf_vals'][:,np.newaxis,:]
plotter_wkde_2.py:41: RuntimeWarning: invalid value encountered in divide
  spatial_KDE_vals = spatial_KDE['pdf_vals']/norm_KDE['pdf_vals'][:,np.newaxis,:]


('found', 53375, 'matching events.')
(373, 'events, or', 0.0069882903981264635, '% are out of plotting range')
(2, 'events, or', 3.7470725995316157e-05, '% are out of histogram bounds')
[9.020549444941356, 8.181864456888885]


/home/ge56lag/py2/lib/python2.7/site-packages/matplotlib/cbook/deprecation.py:107: MatplotlibDeprecationWarning: Adding an axes using the same arguments as a previous axes currently reuses the earlier instance.  In a future version, a new instance will always be created and returned.  Meanwhile, this warning can be suppressed, and the future behavior ensured, by passing a unique label to each axes instance.
  warnings.warn(message, mplDeprecation, stacklevel=1)
/home/ge56lag/py2/lib/python2.7/site-packages/matplotlib/font_manager.py:1331: UserWarning: findfont: Font family [u'sans-serif'] not found. Falling back to DejaVu Sans
  (prop.get_family(), self.defaultFamily[fontext]))


### Hans part

In [ ]:
plotter_wkde = importlib.import_module('plotter_wkde_2')

gammas = [1.5, 2.0, 2.5, 3.0]

xpts = np.array([2.5 + i*0.5 for i in range(9)])
ypts = np.array([1.4, 1.1, 0.9, 0.8, 0.7, 0.6, 0.5, 0.4, 0.4])
tymin = 0.1

base_path = '/home/ge56lag'
output_dir = 'output_Hans/RGI'

for gamma in gammas:
    spatial_KDE_path = '{}/Software/KDE/output/sig_psi_E/adaptive_kd_plotter_wkde_gamma_{}_phi0_1.0/pdf/sig_psi_E.pkl'.format(base_path, gamma)
    norm_KDE_path = '{}/Software/KDE/output/sig_E/adaptive_kd_plotter_wkde_gamma_{}_phi0_1.0/pdf/sig_E.pkl'.format(base_path, gamma)

    spatial_pdf, mc = plotter_wkde.setup(base_path, spatial_KDE_path, norm_KDE_path)

    mc = diffuse_cuts(mc)
    
    for obj in zip(xpts, ypts):
        logE, tymax = obj
        for sigma_p in np.linspace(tymin, tymax, 10):
            try:
                plotter_wkde.make_plot(spatial_pdf, mc, logE, sigma_p, gamma=gamma, delta_sigma=0.2, show_quantile=False, output_dir=output_dir)
            except ZeroDivisionError:
                print("ZeroDivisionError for:")
                print(gamma, logE, sigma_p)

('found', 0, 'matching events.')
ZeroDivisionError for:
(1.5, 2.5, 0.1)
('found', 4991, 'matching events.')
(60, 'events, or', 0.012021638950110198, '% are out of plotting range')
(0, 'events, or', 0.0, '% are out of histogram bounds')
('found', 35557, 'matching events.')
(276, 'events, or', 0.007762184661248137, '% are out of plotting range')
(4, 'events, or', 0.0001124954298731614, '% are out of histogram bounds')
[8.41098305781426, 7.650794560654021, 6.501607992653313, 6.307889601641824]
('found', 56738, 'matching events.')
(416, 'events, or', 0.007331946843385386, '% are out of plotting range')
(2, 'events, or', 3.5249744439352814e-05, '% are out of histogram bounds')
[12.617510909314428, 9.020549444941356]
('found', 63773, 'matching events.')
(495, 'events, or', 0.007761905508600819, '% are out of plotting range')
(2, 'events, or', 3.136123437818513e-05, '% are out of histogram bounds')
[12.617510909314428, 11.033822174193403]
('found', 62631, 'matching events.')
(521, 'events, or

('found', 50, 'matching events.')
(1, 'events, or', 0.02, '% are out of plotting range')
(0, 'events, or', 0.0, '% are out of histogram bounds')
('found', 7868, 'matching events.')
(73, 'events, or', 0.009278088459583122, '% are out of plotting range')
(10, 'events, or', 0.0012709710218607015, '% are out of histogram bounds')
[4.6466770580664365, 3.8192629093464876, 3.5987039114939168, 3.3670233818814865, 3.2845311804892217, 3.190977101428996, 3.142361449317129, 3.113263594620244, 3.101622991832853, 3.0962347036486615]
('found', 21008, 'matching events.')
(274, 'events, or', 0.013042650418888043, '% are out of plotting range')
(26, 'events, or', 0.0012376237623762376, '% are out of histogram bounds')
[12.41051144890517, 8.058151642588475, 7.906529479385936, 7.462933518429186, 6.59116314540087, 6.366129575696905, 5.930486605861951, 5.773532192343948, 5.599477103069061, 5.553318642295504, 5.234536466673288, 5.176637283765948, 5.04489277388596, 5.008102280586733, 4.990096269348767, 4.9397

('found', 24292, 'matching events.')
(617, 'events, or', 0.025399308414292773, '% are out of plotting range')
(53, 'events, or', 0.0021817882430429773, '% are out of histogram bounds')
[24.83745964717852, 22.75691763111164, 21.714600812214705, 21.04898954533904, 19.85617432409669, 19.016364045871335, 18.712387703161937, 18.592916487719137, 18.583967916636464, 17.971129930126292, 17.919870645336502, 17.911647748368914, 17.874014698808143, 17.582718000794827, 17.488785313940937, 17.23330594416043, 17.2054279440959, 17.137259833448056, 16.99392863501055, 16.97065289911932, 16.84157193520494, 16.666319961330366, 16.624106238202668, 16.49508204955689, 16.488426600608108, 16.242858502598814, 16.233621016319322, 16.08454356579494, 15.965501122115704, 15.835125580919048, 15.752384762574222, 15.360787908077349, 15.277116695506274, 15.254406044253548, 15.249766473006556, 15.206511147841326, 15.148243166753955, 15.09132989275273, 15.011098775386222, 14.98249185136185, 14.806506913246805, 14.76421

('found', 6651, 'matching events.')
(226, 'events, or', 0.03397985265373628, '% are out of plotting range')
(57, 'events, or', 0.008570139828597204, '% are out of histogram bounds')
[38.82290013116716, 37.93540066221075, 29.024921010733, 27.32816174611634, 26.381034912710597, 25.405242672283805, 25.281498160350463, 24.885511573091534, 24.841473072668343, 23.26306139234109, 23.07535896714426, 23.024654439063973, 22.268223674420508, 20.691078771108927, 19.95477552512337, 19.72933172075412, 18.888425179609232, 18.88536959161304, 18.4686613904932, 18.132904644220794, 17.443584626650534, 16.839458832184917, 16.161933051223336, 16.033538720024428, 15.999429716338435, 15.859078133469927, 15.818729086890917, 15.758168468073068, 15.727429106734103, 15.698698280198597, 15.656934086393903, 15.413964829218775, 15.347969994939572, 15.082330166584127, 15.022633105512874, 14.956856461019202, 14.905310373897567, 14.841418616896158, 14.425547482359399, 14.424516730020708, 14.109876194495904, 14.0502500

('found', 2752, 'matching events.')
(3, 'events, or', 0.0010901162790697674, '% are out of plotting range')
(0, 'events, or', 0.0, '% are out of histogram bounds')
('found', 5180, 'matching events.')
(20, 'events, or', 0.003861003861003861, '% are out of plotting range')
(7, 'events, or', 0.0013513513513513514, '% are out of histogram bounds')
[4.579782377346181, 3.8942910974896874, 3.8498974700042443, 3.734534285544038, 3.603425070698006, 2.6623144651475386, 2.615604451487949]
('found', 4279, 'matching events.')
(32, 'events, or', 0.007478382799719561, '% are out of plotting range')
(12, 'events, or', 0.002804393549894835, '% are out of histogram bounds')
[7.385369516401584, 6.469623938122343, 5.659794570203385, 4.569886137436012, 4.534870645281853, 4.3740995809157095, 3.9522017326966568, 3.8942910974896874, 3.7172403012376063, 3.630034583689876, 3.5987700103338924, 3.44914319719314]
('found', 3454, 'matching events.')
(44, 'events, or', 0.012738853503184714, '% are out of plotting ra

...

('found', 94842, 'matching events.')
(1470, 'events, or', 0.015499462263554122, '% are out of plotting range')
(36, 'events, or', 0.0003795786676788764, '% are out of histogram bounds')
[19.822549275058538, 15.963850616210664, 13.8971006390324, 13.850260670650973, 13.74364709242348, 13.581572537881028, 13.48000171936888, 12.7760608582032, 12.593899157759711, 12.39925162578391, 12.346443743997563, 12.297932152245206, 12.21546454245379, 12.171083466126463, 11.9889570132379, 11.936470173275053, 11.799195823030887, 11.79036654104307, 11.744522909147092, 11.64881672064898, 11.629170619213058, 11.50074347895268, 11.412302131137547, 11.385500028686002, 11.293890465100406, 11.068620002397935, 10.83743055507514, 10.776571031195985, 10.718263890966512, 10.61937807149346, 10.57172896213412, 10.530780753900629, 10.530541365200138, 10.520147761010668, 10.517671663029732, 10.507293175192011]
('found', 94761, 'matching events.')
(1575, 'events, or', 0.01662076170576503, '% are out of plotting range')

('found', 26422, 'matching events.')
(574, 'events, or', 0.021724320641889335, '% are out of plotting range')
(56, 'events, or', 0.0021194459162818865, '% are out of histogram bounds')
[17.582718000794827, 16.619957151119326, 16.48416588342359, 16.242858502598814, 16.224119601984807, 15.804863627639634, 15.211408080142153, 13.929170328561256, 13.723635934311325, 13.694920005748253, 13.495821742433119, 13.35477926208236, 13.215426033638808, 13.177367610857454, 13.059058627393735, 13.045818771339611, 13.016755257029912, 12.80494885879274, 12.719595425616008, 12.693643330743349, 12.18464688171545, 12.023416720824615, 11.786218610161129, 11.745205848738419, 11.728508155758231, 11.703261000254491, 11.552908580367825, 11.489175163361493, 11.476244628659654, 11.438965983953327, 11.274789457535878, 11.208458857777435, 11.170508031154773, 11.07602073585583, 11.046173153662673, 10.951914330323518, 10.940987559374118, 10.912880812471636, 10.839263769414789, 10.771391780585851, 10.721013903997878,

('found', 8099, 'matching events.')
(171, 'events, or', 0.021113717742931228, '% are out of plotting range')
(48, 'events, or', 0.00592665761205087, '% are out of histogram bounds')
[25.59458158167593, 16.797241112016234, 16.57651701077771, 16.51548784801882, 15.277576479497926, 14.531262939175702, 13.339304268888771, 12.148616511830747, 12.057066739553132, 11.977324260903, 11.82468031252229, 11.403488789536828, 11.311446247047353, 11.085456657693951, 11.070555622691487, 11.069824913262249, 10.960010088732094, 10.69331704944541, 10.661991566808979, 10.609287748052237, 10.525721554790081, 10.379785350434794, 10.000691503775283, 9.80194518679772, 9.73501728261468, 9.70460126019836, 9.689037736951223, 9.685221214206674, 9.53668055939448, 9.511763405369557, 9.393481381531364, 9.377842390290942, 9.374175296207348, 9.186706147055848, 9.083280756600862, 9.015615981570127, 8.828617681973615, 8.754863075765073, 8.694682623528275, 8.642487622207973, 8.50442625762338, 8.442293261657264, 8.4129472

('found', 3944, 'matching events.')
(97, 'events, or', 0.024594320486815417, '% are out of plotting range')
(44, 'events, or', 0.011156186612576065, '% are out of histogram bounds')
[16.669430066521798, 15.301129307841157, 15.081329353754574, 14.974294788704423, 14.34935831979691, 13.504662683761786, 13.03397538830521, 11.765754489749671, 11.514143617059002, 11.35453465501741, 10.694508332596941, 10.0769889184913, 9.999926846395105, 9.997384627551744, 9.899048034524585, 9.83657438391031, 9.80229643061562, 9.586478432286897, 9.398566174495429, 9.140703338543425, 9.136367998333872, 8.841153768443661, 8.821357472402875, 8.486229572639633, 8.393503007908986, 8.281210646785654, 8.24803743660502, 8.11963851669104, 8.07599493084697, 8.057612504446475, 7.753389291467325, 7.747865729322058, 7.670446339107755, 7.64511743205974, 7.636303776894449, 7.576483254946728, 7.415850562014342, 7.275545898775509, 7.2326107269355795, 7.209893587812909, 7.183966536301049, 7.146600721489216, 7.093821364935488

('found', 1726, 'matching events.')
(60, 'events, or', 0.03476245654692932, '% are out of plotting range')
(26, 'events, or', 0.015063731170336037, '% are out of histogram bounds')
[28.168020940550576, 19.375654536242852, 18.059447442548986, 16.85925678867748, 16.144713799650557, 15.648561793416237, 14.847630898297473, 14.666074564954068, 13.195149725330488, 12.33785465902806, 12.245812771723713, 11.978179039375858, 11.806444855090122, 11.363457280721136, 11.188479279157683, 11.178489824149828, 10.940686125053487, 10.874312311507364, 10.787203243895455, 9.732010986091629, 9.5497186613012, 9.072988404903485, 8.243287648957837, 8.233434843751802, 7.965766367360296, 7.866923786430033]
('found', 1503, 'matching events.')
(54, 'events, or', 0.03592814371257485, '% are out of plotting range')
(21, 'events, or', 0.013972055888223553, '% are out of histogram bounds')
[28.168020940550576, 19.375654536242852, 18.059447442548986, 16.144713799650557, 15.648561793416237, 14.847630898297473, 14.6660